<a href="https://colab.research.google.com/github/srirambandi/compsci685/blob/main/train_and_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch transformers datasets sympy tqdm scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
DATA_PATH = "fin_dataset.csv"
cols = ["id", "equ_str", "equ_prefix", "sol_str", "sol_prefix"]

df = pd.read_csv(DATA_PATH, header=0, names=cols)
print(f"Total examples: {len(df)}")
df.head(5)

Total examples: 33874


,id,equ_str,equ_prefix,sol_str,sol_prefix
0,0,x^4*e^(x) + 4*x^3*e^(x) + y' - 1 = 0,add add add -1 mul pow x 4 exp x mul mul 4 pow...,c - x^4*e^(x) + x,add add c x mul mul -1 pow x 4 exp x
1,1,-2*y*ln(12) + y' = 0,add mul mul -2 y log 12 y',12^(c + 2*x),pow 12 add c mul 2 x
2,2,2*x + 9*y' - 9 = 0,add add -9 mul 2 x mul 9 y',c - x^2/9 + x,add add c x mul div -1 9 pow x 2
3,3,2*x^2 + x*y' - y = 0,add add mul -1 y mul 2 pow x 2 mul x y',c*x + x*(4 - 2*x) + x,add add x mul c x mul x add 4 mul -2 x
4,4,54*x*e^(y/9) + y' = 0,add mul mul 54 x exp mul div 1 9 y y',-9*ln(c + 3*x^2),mul -9 log add c mul 3 pow x 2


In [5]:
# stats about our dataset
df["equ_len"] = df["equ_prefix"].str.split().apply(len)
df["sol_len"] = df["sol_prefix"].str.split().apply(len)

df[["equ_len", "sol_len"]].describe().T

,count,mean,std,min,25%,50%,75%,max
equ_len,33874.0,13.184684,7.748491,1.0,8.0,11.0,15.0,115.0
sol_len,33874.0,10.013107,2.727231,2.0,8.0,10.0,11.0,28.0


In [6]:
# first split off test
train_val, test = train_test_split(df, test_size=0.10, random_state=42)
# then split train/val
train, val = train_test_split(train_val, test_size=0.1111, random_state=42)
# (0.1111*0.90 ≈ 0.10 total)

print("Sizes → train:", len(train), "val:", len(val), "test:", len(test))

Sizes → train: 27099 val: 3387 test: 3388


In [7]:
os.makedirs("splits", exist_ok=True)
train.to_csv("splits/train.csv", index=False)
val.to_csv("splits/val.csv", index=False)
test.to_csv("splits/test.csv", index=False)